In [1]:
# pip install -r requirements.txt
# !pip install keras --upgrade

In [2]:
# pip install tensorboard==2.19

In [2]:
# pip install numpy==1.26.4


In [1]:
import json
import pandas as pd
import re
import emoji
import nltk
import string
from spellchecker import SpellChecker
import json
from collections import Counter
from textstat import textstat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.feature_selection import SelectFromModel
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import concatenate_datasets
import evaluate
from transformers import DataCollatorForSeq2Seq
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline
from random import randrange
from nltk.tokenize import sent_tokenize


# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
# nltk.download('all')

/Users/admed/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package punkt to /Users/admed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/admed/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
root_path = 'corpus/'
path='corpus/data-processing/'

In [5]:
# Load the dataset
with open(root_path + 'train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
with open(root_path + 'test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
with open(root_path + 'val.json', 'r', encoding='utf-8') as f:
    eval_data = json.load(f)

# Convert to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
eval_df = pd.DataFrame(eval_data)

# Combine all three into one
full_df = pd.concat([train_df, eval_df], ignore_index=True)

In [5]:
# Initialize spell checker
spell = SpellChecker()





# Basic text preprocessing functions (as defined before)
def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')

def remove_emoticons(text):
    # 1. Remove common kaomojis/emoticons with special chars inside (), [], {}, or without
    kaomoji_pattern = r"""
        (?:
            [\(\[\{]        # opening bracket
            [^\w\s]{2,}     # at least 2 non-alphanumeric, non-space chars
            [\)\]\}]        # closing bracket
        )
        |
        (?:
            (?<!\w)         # not part of a word
            [<>]?
            [:;=8xX]        # eyes
            [-'^]?
            [\)\(DOpPb3/\|] # mouth types
            (?!\w)
        )
        |
        (?:
            (?<!\w)
            [\^*=_~][\^*=_~\.]{1,}[^\w\s]?
            (?!\w)
        )
    """
    text = re.sub(kaomoji_pattern, '', text, flags=re.VERBOSE)

    # 2. Remove leftover arm-like patterns like \ / or / \ or \_/
    arms_pattern = r'(\\|\/)[\s_•◡\.]+(\\|\/)'
    text = re.sub(arms_pattern, '', text)

    # 3. Remove lone slashes and backslashes left over
    text = re.sub(r'\s*[\\/]+\s*', ' ', text)

    # 4. Normalize spacing
    text = re.sub(r'\s{2,}', ' ', text).strip()

    return text

def clean_mixed_punctuation(text):
    return re.sub(r'([!?.,]){3,}', lambda m: m.group(0)[-1], text)


def correct_spelling(text):
    words = text.split()
    corrected_words = []

    for word in words:
        # Skip correction for very short words, placeholders, or common chat terms
        if len(word) <= 2 or word.startswith('[') or word in {'im', 'ok', 'lol', 'u', 'ur', 'r'}:
            corrected_words.append(word)
            continue

        # Extract leading/trailing punctuation using regex
        match = re.match(r"^(\W*)(\w+)(\W*)$", word)
        if match:
            leading_punct, core_word, trailing_punct = match.groups()
        else:
            # In case the word doesn't match the pattern, treat it as-is
            corrected_words.append(word)
            continue

        # Check spelling of the core word
        if core_word.lower() in spell:
            corrected_core = core_word
        else:
            correction = spell.correction(core_word)
            corrected_core = correction if correction else core_word

        # Reconstruct the word with original punctuation
        corrected_word = f"{leading_punct}{corrected_core}{trailing_punct}"
        corrected_words.append(corrected_word)

    return ' '.join(corrected_words)


# Function to clean text with all preprocessing steps
def clean_text(text):
    # Remove special characters like \r\n
    text = re.sub(r'\r\n', ' ', text)

    # Replace file indicators with placeholders
    text = re.sub(r'<file_photo>', '', text)
    text = re.sub(r'<file_video>', '', text)
    text = re.sub(r'<file_gif>', '', text)
    text = re.sub(r'<file_other>', '', text)
    # print("Text: "+text)

    # Remove emojis
    text = remove_emoji(text)

    # Remove emoticons
    text = remove_emoticons(text)

    # Remove extra punctuation
    text = clean_mixed_punctuation(text)

    # Correct spelling
    text = correct_spelling(text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Main function to process a dialogue
def process_dialogue(dialogue_obj):
    dialogue = dialogue_obj['dialogue']
    
    # Clean the dialogue
    clean_dialogue = clean_text(dialogue)
    
    return {
        'id': dialogue_obj.get('id', ''),
        'summary': dialogue_obj.get('summary', ''),
        'dialogue': clean_dialogue
    }

# Process the entire dataset
processed_dialogues = []
for i, row in full_df.iterrows():
    try:
        if(len(row['dialogue'])>0):
            processed = process_dialogue(row)
            processed_dialogues.append(processed)
            print(f"Processed dialogue {i+1}/{len(full_df)}")
    except Exception as e:
        print(f"Error processing dialogue {i+1}: {e}")



# Save the segmented conversations
with open(path+'cleaned_train_conversations.json', 'w', encoding='utf-8') as f:
    json.dump(processed_dialogues, f, indent=4, ensure_ascii=False)

# Display a sample segmented conversation
def display_sample_segmentation(processed, idx=0):
    print(f"Original Summary: {processed[idx]['summary']}")
    print("Dialogue:")
    print(processed[idx]['dialogue'])

# Display a sample
if processed_dialogues:
    display_sample_segmentation(processed_dialogues)

Processed dialogue 1/15550
Processed dialogue 2/15550
Processed dialogue 3/15550
Processed dialogue 4/15550
Processed dialogue 5/15550
Processed dialogue 6/15550
Processed dialogue 7/15550
Processed dialogue 8/15550
Processed dialogue 9/15550
Processed dialogue 10/15550
Processed dialogue 11/15550
Processed dialogue 12/15550
Processed dialogue 13/15550
Processed dialogue 14/15550
Processed dialogue 15/15550
Processed dialogue 16/15550
Processed dialogue 17/15550
Processed dialogue 18/15550
Processed dialogue 19/15550
Processed dialogue 20/15550
Processed dialogue 21/15550
Processed dialogue 22/15550
Processed dialogue 23/15550
Processed dialogue 24/15550
Processed dialogue 25/15550
Processed dialogue 26/15550
Processed dialogue 27/15550
Processed dialogue 28/15550
Processed dialogue 29/15550
Processed dialogue 30/15550
Processed dialogue 31/15550
Processed dialogue 32/15550
Processed dialogue 33/15550
Processed dialogue 34/15550
Processed dialogue 35/15550
Processed dialogue 36/15550
P

Processed dialogue 301/15550
Processed dialogue 302/15550
Processed dialogue 303/15550
Processed dialogue 304/15550
Processed dialogue 305/15550
Processed dialogue 306/15550
Processed dialogue 307/15550
Processed dialogue 308/15550
Processed dialogue 309/15550
Processed dialogue 310/15550
Processed dialogue 311/15550
Processed dialogue 312/15550
Processed dialogue 313/15550
Processed dialogue 314/15550
Processed dialogue 315/15550
Processed dialogue 316/15550
Processed dialogue 317/15550
Processed dialogue 318/15550
Processed dialogue 319/15550
Processed dialogue 320/15550
Processed dialogue 321/15550
Processed dialogue 322/15550
Processed dialogue 323/15550
Processed dialogue 324/15550
Processed dialogue 325/15550
Processed dialogue 326/15550
Processed dialogue 327/15550
Processed dialogue 328/15550
Processed dialogue 329/15550
Processed dialogue 330/15550
Processed dialogue 331/15550
Processed dialogue 332/15550
Processed dialogue 333/15550
Processed dialogue 334/15550
Processed dial

Processed dialogue 587/15550
Processed dialogue 588/15550
Processed dialogue 589/15550
Processed dialogue 590/15550
Processed dialogue 591/15550
Processed dialogue 592/15550
Processed dialogue 593/15550
Processed dialogue 594/15550
Processed dialogue 595/15550
Processed dialogue 596/15550
Processed dialogue 597/15550
Processed dialogue 598/15550
Processed dialogue 599/15550
Processed dialogue 600/15550
Processed dialogue 601/15550
Processed dialogue 602/15550
Processed dialogue 603/15550
Processed dialogue 604/15550
Processed dialogue 605/15550
Processed dialogue 606/15550
Processed dialogue 607/15550
Processed dialogue 608/15550
Processed dialogue 609/15550
Processed dialogue 610/15550
Processed dialogue 611/15550
Processed dialogue 612/15550
Processed dialogue 613/15550
Processed dialogue 614/15550
Processed dialogue 615/15550
Processed dialogue 616/15550
Processed dialogue 617/15550
Processed dialogue 618/15550
Processed dialogue 619/15550
Processed dialogue 620/15550
Processed dial

Processed dialogue 875/15550
Processed dialogue 876/15550
Processed dialogue 877/15550
Processed dialogue 878/15550
Processed dialogue 879/15550
Processed dialogue 880/15550
Processed dialogue 881/15550
Processed dialogue 882/15550
Processed dialogue 883/15550
Processed dialogue 884/15550
Processed dialogue 885/15550
Processed dialogue 886/15550
Processed dialogue 887/15550
Processed dialogue 888/15550
Processed dialogue 889/15550
Processed dialogue 890/15550
Processed dialogue 891/15550
Processed dialogue 892/15550
Processed dialogue 893/15550
Processed dialogue 894/15550
Processed dialogue 895/15550
Processed dialogue 896/15550
Processed dialogue 897/15550
Processed dialogue 898/15550
Processed dialogue 899/15550
Processed dialogue 900/15550
Processed dialogue 901/15550
Processed dialogue 902/15550
Processed dialogue 903/15550
Processed dialogue 904/15550
Processed dialogue 905/15550
Processed dialogue 906/15550
Processed dialogue 907/15550
Processed dialogue 908/15550
Processed dial

Processed dialogue 1159/15550
Processed dialogue 1160/15550
Processed dialogue 1161/15550
Processed dialogue 1162/15550
Processed dialogue 1163/15550
Processed dialogue 1164/15550
Processed dialogue 1165/15550
Processed dialogue 1166/15550
Processed dialogue 1167/15550
Processed dialogue 1168/15550
Processed dialogue 1169/15550
Processed dialogue 1170/15550
Processed dialogue 1171/15550
Processed dialogue 1172/15550
Processed dialogue 1173/15550
Processed dialogue 1174/15550
Processed dialogue 1175/15550
Processed dialogue 1176/15550
Processed dialogue 1177/15550
Processed dialogue 1178/15550
Processed dialogue 1179/15550
Processed dialogue 1180/15550
Processed dialogue 1181/15550
Processed dialogue 1182/15550
Processed dialogue 1183/15550
Processed dialogue 1184/15550
Processed dialogue 1185/15550
Processed dialogue 1186/15550
Processed dialogue 1187/15550
Processed dialogue 1188/15550
Processed dialogue 1189/15550
Processed dialogue 1190/15550
Processed dialogue 1191/15550
Processed 

...

Processed dialogue 14376/15550
Processed dialogue 14377/15550
Processed dialogue 14378/15550
Processed dialogue 14379/15550
Processed dialogue 14380/15550
Processed dialogue 14381/15550
Processed dialogue 14382/15550
Processed dialogue 14383/15550
Processed dialogue 14384/15550
Processed dialogue 14385/15550
Processed dialogue 14386/15550
Processed dialogue 14387/15550
Processed dialogue 14388/15550
Processed dialogue 14389/15550
Processed dialogue 14390/15550
Processed dialogue 14391/15550
Processed dialogue 14392/15550
Processed dialogue 14393/15550
Processed dialogue 14394/15550
Processed dialogue 14395/15550
Processed dialogue 14396/15550
Processed dialogue 14397/15550
Processed dialogue 14398/15550
Processed dialogue 14399/15550
Processed dialogue 14400/15550
Processed dialogue 14401/15550
Processed dialogue 14402/15550
Processed dialogue 14403/15550
Processed dialogue 14404/15550
Processed dialogue 14405/15550
Processed dialogue 14406/15550
Processed dialogue 14407/15550
Processe

Processed dialogue 14642/15550
Processed dialogue 14643/15550
Processed dialogue 14644/15550
Processed dialogue 14645/15550
Processed dialogue 14646/15550
Processed dialogue 14647/15550
Processed dialogue 14648/15550
Processed dialogue 14649/15550
Processed dialogue 14650/15550
Processed dialogue 14651/15550
Processed dialogue 14652/15550
Processed dialogue 14653/15550
Processed dialogue 14654/15550
Processed dialogue 14655/15550
Processed dialogue 14656/15550
Processed dialogue 14657/15550
Processed dialogue 14658/15550
Processed dialogue 14659/15550
Processed dialogue 14660/15550
Processed dialogue 14661/15550
Processed dialogue 14662/15550
Processed dialogue 14663/15550
Processed dialogue 14664/15550
Processed dialogue 14665/15550
Processed dialogue 14666/15550
Processed dialogue 14667/15550
Processed dialogue 14668/15550
Processed dialogue 14669/15550
Processed dialogue 14670/15550
Processed dialogue 14671/15550
Processed dialogue 14672/15550
Processed dialogue 14673/15550
Processe

Processed dialogue 14907/15550
Processed dialogue 14908/15550
Processed dialogue 14909/15550
Processed dialogue 14910/15550
Processed dialogue 14911/15550
Processed dialogue 14912/15550
Processed dialogue 14913/15550
Processed dialogue 14914/15550
Processed dialogue 14915/15550
Processed dialogue 14916/15550
Processed dialogue 14917/15550
Processed dialogue 14918/15550
Processed dialogue 14919/15550
Processed dialogue 14920/15550
Processed dialogue 14921/15550
Processed dialogue 14922/15550
Processed dialogue 14923/15550
Processed dialogue 14924/15550
Processed dialogue 14925/15550
Processed dialogue 14926/15550
Processed dialogue 14927/15550
Processed dialogue 14928/15550
Processed dialogue 14929/15550
Processed dialogue 14930/15550
Processed dialogue 14931/15550
Processed dialogue 14932/15550
Processed dialogue 14933/15550
Processed dialogue 14934/15550
Processed dialogue 14935/15550
Processed dialogue 14936/15550
Processed dialogue 14937/15550
Processed dialogue 14938/15550
Processe

Processed dialogue 15172/15550
Processed dialogue 15173/15550
Processed dialogue 15174/15550
Processed dialogue 15175/15550
Processed dialogue 15176/15550
Processed dialogue 15177/15550
Processed dialogue 15178/15550
Processed dialogue 15179/15550
Processed dialogue 15180/15550
Processed dialogue 15181/15550
Processed dialogue 15182/15550
Processed dialogue 15183/15550
Processed dialogue 15184/15550
Processed dialogue 15185/15550
Processed dialogue 15186/15550
Processed dialogue 15187/15550
Processed dialogue 15188/15550
Processed dialogue 15189/15550
Processed dialogue 15190/15550
Processed dialogue 15191/15550
Processed dialogue 15192/15550
Processed dialogue 15193/15550
Processed dialogue 15194/15550
Processed dialogue 15195/15550
Processed dialogue 15196/15550
Processed dialogue 15197/15550
Processed dialogue 15198/15550
Processed dialogue 15199/15550
Processed dialogue 15200/15550
Processed dialogue 15201/15550
Processed dialogue 15202/15550
Processed dialogue 15203/15550
Processe

Processed dialogue 15441/15550
Processed dialogue 15442/15550
Processed dialogue 15443/15550
Processed dialogue 15444/15550
Processed dialogue 15445/15550
Processed dialogue 15446/15550
Processed dialogue 15447/15550
Processed dialogue 15448/15550
Processed dialogue 15449/15550
Processed dialogue 15450/15550
Processed dialogue 15451/15550
Processed dialogue 15452/15550
Processed dialogue 15453/15550
Processed dialogue 15454/15550
Processed dialogue 15455/15550
Processed dialogue 15456/15550
Processed dialogue 15457/15550
Processed dialogue 15458/15550
Processed dialogue 15459/15550
Processed dialogue 15460/15550
Processed dialogue 15461/15550
Processed dialogue 15462/15550
Processed dialogue 15463/15550
Processed dialogue 15464/15550
Processed dialogue 15465/15550
Processed dialogue 15466/15550
Processed dialogue 15467/15550
Processed dialogue 15468/15550
Processed dialogue 15469/15550
Processed dialogue 15470/15550
Processed dialogue 15471/15550
Processed dialogue 15472/15550
Processe

In [6]:
with open(root_path + 'train.json', 'r', encoding='utf-8') as f:
    processed_dialogues = json.load(f)

In [7]:
# model_id="google/flan-t5-base"
model_id="t5-small"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

/Users/admed/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
from datasets import Dataset

# Convert the list of dicts into a Dataset
processed_dialogues_dataset = Dataset.from_list(processed_dialogues)


# The maximum total input sequence length after tokenization.
tokenized_inputs = processed_dialogues_dataset.map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
tokenized_targets = processed_dialogues_dataset.map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

  0%|          | 0/15 [00:00<?, ?ba/s]

Max source length: 512


  0%|          | 0/15 [00:00<?, ?ba/s]

Max target length: 94


In [9]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = processed_dialogues_dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset.features)}")

  0%|          | 0/15 [00:00<?, ?ba/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [10]:
from transformers import AutoModelForSeq2SeqLM

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [11]:
# Metric
# metric = evaluate.load("rouge")
from datasets import load_metric
metric = load_metric("rouge")


# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {k: round(v * 100, 4) for k, v in result.items()}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     return result

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    new_result = {}
    for k, v in result.items():
        if isinstance(v, dict):
            # dict like {'precision':..., 'recall':..., 'fmeasure':...}
            f1 = v.get("fmeasure", 0.0)
            new_result[k] = round(float(f1) * 100, 4)
        elif isinstance(v, (tuple, list)):
            try:
#                 f1 = v[2]
#                 new_result[k] = round(float(f1) * 100, 4)
                new_result[k] = round(v.mid.fmeasure * 100, 4)
            except (IndexError, TypeError):
                new_result[k] = 0.0
        elif isinstance(v, (float, int)):
            new_result[k] = round(float(v) * 100, 4)
        else:
            # fallback: unknown structure
            new_result[k] = 0.0

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    new_result["gen_len"] = np.mean(prediction_lens)

    return new_result



/var/folders/2k/bnb3_9y57s1gv8yjc7f8nlkw0000gn/T/ipykernel_37768/1823827894.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [12]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


In [13]:
# Split into 80% train and 20% validation
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Get train and validation sets
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [14]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=path,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    learning_rate=5e-5,
    num_train_epochs=10,
    logging_dir=f"{path}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

/Users/admed/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.033700,1.808667,40.916300,17.601500,33.929900,37.646800,16.647099
2,1.921800,1.767065,41.384000,18.173700,34.499800,38.152200,16.806244
3,1.843000,1.736831,42.592500,18.921100,35.414400,39.043500,16.663386
4,1.809300,1.718584,43.034200,19.205700,35.605800,39.378100,16.758738
5,1.763400,1.714216,42.766600,19.230200,35.511200,39.204400,16.794028
6,1.744200,1.703531,43.323000,19.860700,36.257300,39.805000,16.699695
7,1.709100,1.699314,43.462900,19.816000,36.150400,39.850800,16.801154
8,1.692100,1.694547,43.689900,19.837800,36.371600,39.985400,16.753987
9,1.693200,1.693743,43.609700,19.759500,36.234800,39.883900,16.810316
10,1.664700,1.693381,43.705000,19.849700,36.302500,39.980300,16.847302


/Users/admed/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/Users/admed/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/Users/admed/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/Users/admed/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the mode

TrainOutput(global_step=14740, training_loss=1.7861574480944444, metrics={'train_runtime': 8122.2168, 'train_samples_per_second': 14.51, 'train_steps_per_second': 1.815, 'total_flos': 1.59500313034752e+16, 'train_loss': 1.7861574480944444, 'epoch': 10.0})

In [16]:
trainer.evaluate()

/Users/admed/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 1.6933808326721191,
 'eval_rouge1': 43.705,
 'eval_rouge2': 19.8497,
 'eval_rougeL': 36.3025,
 'eval_rougeLsum': 39.9803,
 'eval_gen_len': 16.8473023413641,
 'eval_runtime': 208.7578,
 'eval_samples_per_second': 14.117,
 'eval_steps_per_second': 1.768,
 'epoch': 10.0}

In [17]:

tokenizer.save_pretrained(path+'/model')
trainer.create_model_card()

# Save the model
# trainer.save_pretrained(path+"/saved_model")
trainer.save_model(path+"/model")


In [19]:
     

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model=path+'/model')

# select a random test sample
sample = test_df.iloc[randrange(len(test_df))]
# print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"Human Summary: \n{sample['summary']}")
# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"t5-small summary:\n{res[0]['summary_text']}")

Human Summary: 
Fatima is worried about Jenson and Alene. Alene has issues. Lincoln doesn't want Fatima to worry about others too much.
t5-small summary:
Fatima talked to Jenson, he's not happy. He thinks Alene may have a depression and needs a doctor. Alene has problems facing it.


In [ ]:
# pip install tf-keras